In [1]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.1 MB/s eta 0:00:00


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
import optuna

# Load data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --------------------------------------------------
# 🔹 1. GridSearchCV
# --------------------------------------------------
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7]
}

grid_model = GridSearchCV(
    RandomForestClassifier(),
    param_grid,
    cv=5,
    scoring='accuracy'
)

grid_model.fit(X_train, y_train)

print("🔍 GridSearchCV Best Params:", grid_model.best_params_)
print("🔍 GridSearchCV Best Score:", grid_model.best_score_)

# --------------------------------------------------
# 🔹 2. RandomizedSearchCV
# --------------------------------------------------
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(2, 10)
}

random_model = RandomizedSearchCV(
    RandomForestClassifier(),
    param_distributions=param_dist,
    n_iter=20,        # number of random combinations
    cv=5,
    random_state=42,
    scoring='accuracy'
)

random_model.fit(X_train, y_train)

print("🎲 RandomizedSearchCV Best Params:", random_model.best_params_)
print("🎲 RandomizedSearchCV Best Score:", random_model.best_score_)

# --------------------------------------------------
# 🔹 3. Optuna
# --------------------------------------------------
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 2, 10)

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    score = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("🧠 Optuna Best Params:", study.best_params)
print("🧠 Optuna Best Score:", study.best_value)


🔍 GridSearchCV Best Params: {'max_depth': 3, 'n_estimators': 50}
🔍 GridSearchCV Best Score: 0.9428571428571428


[I 2025-07-29 03:47:20,189] A new study created in memory with name: no-name-33a403fa-e4f5-43ff-bb06-01ce17e1cbb4


🎲 RandomizedSearchCV Best Params: {'max_depth': 8, 'n_estimators': 142}
🎲 RandomizedSearchCV Best Score: 0.9428571428571428


[I 2025-07-29 03:47:20,918] Trial 0 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 100, 'max_depth': 9}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-07-29 03:47:21,905] Trial 1 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 136, 'max_depth': 6}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-07-29 03:47:22,739] Trial 2 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 115, 'max_depth': 4}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-07-29 03:47:23,651] Trial 3 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 126, 'max_depth': 7}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-07-29 03:47:24,870] Trial 4 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 171, 'max_depth': 10}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-07-29 03:47:25,716] Trial 5 finished with value: 0.9428571428571428 and parameters: {'n_estimato

🧠 Optuna Best Params: {'n_estimators': 100, 'max_depth': 9}
🧠 Optuna Best Score: 0.9428571428571428
